# How to Query OpenTargets Genetics for L2G Scores

### 1. Create Google Cloud Credentials

Note that the first 1TB of querying is free, but then a fee applies after that, so you will need to link payment information to your queries.  Unless you are doing very large queries, you should not reach the 1TB limit.

1. Go to https://console.cloud.google.com/
2. If you do not have a project already, create one.
3. Go to https://console.cloud.google.com/iam-admin/serviceaccounts & click "+ CREATE SERVICE ACCOUNT"
4. Enter a descriptive Service account name like "groberts-query-opentargets" & set the role to "owner"
5. Once back on the Service Accounts main page, click the "Actions" menu button > "Manage Keys"
6. "ADD KEY" > "Create New Key" > "JSON". A .json file will automatically download. **Be careful with your .json. You will need to call this file, but DO NOT upload it to GitHub as these are your private credentials and malicious actors could use your credentials to rack up expensive queries**

In [1]:
#setup credentials for BigQuery

from google.cloud import bigquery
credentials_path = '/home/robertg1/.ssh/test-bigquery-ot-956f8a01208f.json'
client = bigquery.Client.from_service_account_json(credentials_path)

## 2. Run the Query

In [2]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Define study IDs to keep
study_ids_to_keep = ["GCST005523"]

# Construct parameterized query
query = '''
WITH ranked_genes AS (
    SELECT locus2gene.study_id, 
        locus2gene.chrom, locus2gene.pos, locus2gene.ref, locus2gene.alt, 
        study_metadata.*, 
        genes.gene_name, locus2gene.y_proba_full_model,
        lead_variants.pval,
        ROW_NUMBER() OVER(PARTITION BY locus2gene.study_id, genes.gene_name ORDER BY lead_variants.pval) AS rn
    
    FROM `bigquery-public-data.open_targets_genetics.locus2gene` AS locus2gene
    
    -- Get GWAS metadata
    INNER JOIN `bigquery-public-data.open_targets_genetics.studies` AS study_metadata
    ON locus2gene.study_id = study_metadata.study_id
    
    -- Get HGNC IDs
    INNER JOIN `bigquery-public-data.open_targets_genetics.genes` AS genes
    ON locus2gene.gene_id = genes.gene_id
    
    -- Get lead variant P-values
    INNER JOIN `bigquery-public-data.open_targets_genetics.variant_disease` AS lead_variants
    ON locus2gene.pos = lead_variants.lead_pos
        AND locus2gene.chrom = lead_variants.lead_chrom
        AND locus2gene.study_id = lead_variants.study_id
    
        -- Optional: Filter on the study IDs in the list study_ids_to_keep
        -- WHERE locus2gene.study_id IN UNNEST(@study_ids_to_keep)
        
        -- Optional: filter on a disease
        -- AND LOWER(study_metadata.trait_reported) LIKE '%coeliac%'
        
        -- Remove the "raw" Neale lab results -- I'm not sure what this is
        AND locus2gene.study_id NOT LIKE '%raw%'
        
        -- Filter to a l2g score threshold
        AND locus2gene.y_proba_full_model > 0.2
        
        -- Filter to number of associated loci of at least 10
        AND study_metadata.num_assoc_loci >= 10
        
        -- Filter to n_cases of at least 3000
        AND study_metadata.n_cases >= 3000
)

SELECT * FROM ranked_genes WHERE rn = 1;
'''

# Set query parameters
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("study_ids_to_keep", "STRING", study_ids_to_keep)
    ]
)

# Run the query
query_job = client.query(query, job_config=job_config)
l2g = query_job.to_dataframe()
l2g.sort_values(by=['study_id', 'pval'])

,study_id,chrom,pos,ref,alt,study_id_1,ancestry_initial,ancestry_replication,n_cases,n_initial,...,has_sumstats,num_assoc_loci,source,trait_reported,trait_efos,trait_category,gene_name,y_proba_full_model,pval,rn
37402,FINNGEN_R6_ABDOM_HERNIA,1,219570796,C,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,ZC3H11B,0.685505,5.520000e-43,1
4564,FINNGEN_R6_ABDOM_HERNIA,7,134907837,G,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,CALD1,0.595480,1.000000e-15,1
37401,FINNGEN_R6_ABDOM_HERNIA,7,134907837,G,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,AGBL3,0.217192,1.000000e-15,1
27797,FINNGEN_R6_ABDOM_HERNIA,2,19576027,C,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,OSR1,0.551340,1.320000e-13,1
12405,FINNGEN_R6_ABDOM_HERNIA,12,77829818,G,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,NAV3,0.732569,7.730000e-11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34037,SAIGE_740_1,1,102938828,T,C,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,COL11A1,0.663457,4.860000e-10,1
30248,SAIGE_740_1,6,75478614,C,A,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,FILIP1,0.754302,4.710000e-09,1
34877,SAIGE_740_1,3,52422957,A,G,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,ITIH4,0.242676,1.420000e-08,1
4958,SAIGE_740_1,6,44991647,A,T,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,RUNX2,0.379980,2.630000e-08,1


In [4]:
l2g.shape

(41686, 26)

In [5]:
l2g.columns

Index(['study_id', 'chrom', 'pos', 'ref', 'alt', 'study_id_1',
       'ancestry_initial', 'ancestry_replication', 'n_cases', 'n_initial',
       'n_replication', 'pmid', 'pub_author', 'pub_date', 'pub_journal',
       'pub_title', 'has_sumstats', 'num_assoc_loci', 'source',
       'trait_reported', 'trait_efos', 'trait_category', 'gene_name',
       'y_proba_full_model', 'pval', 'rn'],
      dtype='object')

In [6]:
l2g.to_csv('OpenTargets_L2G_noQC.csv.gz', compression="gzip")